In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import chess
import chess.pgn
import chess.engine
from stockfish import Stockfish
import matplotlib.pyplot as plt
import os

In [10]:
# Define la red neuronal convolucional con Dropout
class ChessCNN(nn.Module):
    def __init__(self):
        super(ChessCNN, self).__init__()
        self.conv1 = nn.Conv2d(13, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.dropout = nn.Dropout(p=0.5)  # Agregamos dropout con probabilidad 0.5
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 4096)  # Aproximadamente el número máximo de movimientos posibles

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 256 * 8 * 8)
        x = self.dropout(torch.relu(self.fc1(x)))  # Aplicamos dropout
        return self.fc2(x)

Prrimera capa de la red : Es una capa convolucional que recibe 13 tableros de ajedrez y constara de 64 neuronas y cada persona solo mirara 3x3 de ese tablero y el padding sirve para que la imagen no cambie a la hora de ser examinada 

Segunda capa de la red : Tambien convolucional y toma de entrada la salida de la otra 

Tercera capa : ahora con 128 neuronas analizando es posible encontrar patrones mas complejos en el tablero de ajedrez

Dropout : Esto sirve para que la neurona no se sobre entrene y sea mas general osea no se fija siempre en el mismo problema , lo que hace es que con una probabilidad del 50 apaga algunas partes para no generar valores tan exactos 

fc1 y fc2 sirven para tomar la desicionm final de aplanando los datos y devolviendo unas 4096 posibilidades 

In [11]:
# Función para convertir el tablero en una matriz para la red neuronal
def board_to_matrix(board):
    matrix = np.zeros((13, 8, 8))
    piece_map = board.piece_map()

    # Piezas en el tablero
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[piece_type + piece_color, row, col] = 1

    # Movimientos legales
    legal_moves = board.legal_moves
    for move in legal_moves:
        row_to, col_to = divmod(move.to_square, 8)
        matrix[12, row_to, col_to] = 1

    return matrix

Esta funcion sirve para tansformar el tablero de chess en un array de 8 x 8 usamos 13 ya que solo las piezas ocupan 12 y 1 para los movimientos legales 

In [12]:
# Crea un input para la red 
def create_input_for_nn(board):
    X = board_to_matrix(board)
    return torch.tensor(X, dtype=torch.float32).unsqueeze(0)

Transforma la matriz en un tensor de pytorch que es lo adecuado para entrenar redes en torch 

In [13]:
def get_engine_evaluation(stockfish, board):
    """
    Devuelve la evaluación actual del tablero desde la perspectiva del motor de ajedrez.
    """
    stockfish.set_fen_position(board.fen())
    evaluation = stockfish.get_evaluation()
    
    if evaluation['type'] == 'cp':
        # Si la evaluación es en centi-pawns
        return evaluation['value'] / 100.0  # Convertimos centi-pawns a valores decimales (por ejemplo, 225 -> 2.25)
    elif evaluation['type'] == 'mate':
        # Si la evaluación es en términos de mate, consideramos un valor muy grande (positivo o negativo)
        return 1000 if evaluation['value'] > 0 else -1000
    else:
        return 0  # En caso de un error

Define las recompensas segun la partida 1 = win -1 = lose  0 = empate 

In [14]:
def save_model(model, optimizer, epoch, path="model_checkpoint.pth"):
    """Guardar el modelo y el optimizador para continuar el entrenamiento."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, path)

def load_model(model, optimizer, path="model_checkpoint.pth"):
    """Cargar el modelo y el optimizador desde un archivo para continuar el entrenamiento."""
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        print(f"Modelo cargado desde {path}, continuar desde la época {epoch+1}")
        return model, optimizer, epoch
    else:
        print("No se encontró ningún checkpoint. Entrenamiento comenzará desde el principio.")
        return model, optimizer, 0

In [15]:
def train_against_stockfish_with_evaluation(model, optimizer, stockfish_path, epochs=10, start_epoch=0):
    stockfish = Stockfish(stockfish_path)
    stockfish.set_depth(15)

    for epoch in range(start_epoch, epochs):
        board = chess.Board()
        game_over = False
        winner = None
        
        move_probabilities_list = []
        chosen_moves_list = []
        evaluations_list = []
        
        pgn_game = chess.pgn.Game()
        node = pgn_game
        
        # Evaluación inicial del tablero
        initial_evaluation = get_engine_evaluation(stockfish, board)
        evaluations_list.append(initial_evaluation)

        while not game_over:
            model.train()  # Modo entrenamiento
            
            board_matrix = create_input_for_nn(board)
            move_probabilities = model(board_matrix).squeeze()
            
            legal_moves = list(board.legal_moves)
            legal_move_probs = [move_probabilities[i].item() for i, move in enumerate(legal_moves)]
            legal_move_indices = list(range(len(legal_moves)))
            
            legal_move_probs = torch.tensor(legal_move_probs)
            
            if len(legal_move_probs) > 0:
                chosen_move_idx = torch.argmax(legal_move_probs).item()
                chosen_move = legal_moves[legal_move_indices[chosen_move_idx]]
                board.push(chosen_move)
                
                move_probabilities_list.append(move_probabilities)
                chosen_moves_list.append(chosen_move_idx)
                
                node = node.add_variation(chosen_move)
                
                # Obtener evaluación después del movimiento
                new_evaluation = get_engine_evaluation(stockfish, board)
                evaluations_list.append(new_evaluation)
                
                # Verificar si el juego ha terminado
                if board.is_game_over():
                    game_over = True
                    break
            else:
                print("No hay movimientos legales disponibles.")
                break

            # Movimiento de Stockfish
            stockfish.set_fen_position(board.fen())
            best_move = stockfish.get_best_move()
            board.push(chess.Move.from_uci(best_move))
            node = node.add_variation(chess.Move.from_uci(best_move))
            
            # Verificar si el juego ha terminado
            if board.is_game_over():
                game_over = True
                break
        
        # Actualizar el modelo basándonos en las evaluaciones
        optimizer.zero_grad()
        
        for i in range(1, len(evaluations_list)):
            reward = evaluations_list[i] - evaluations_list[i - 1]  # Diferencia en la evaluación
            if reward == 0:
                continue  # No hacemos nada si la evaluación no cambió
            
            loss = -torch.log(move_probabilities_list[i-1][chosen_moves_list[i-1]]) * reward
            loss.backward()
        
        optimizer.step()
        
        # Guardar el modelo después de cada época
        save_model(model, optimizer, epoch)
        
        # Guardar la partida en formato PGN dentro de la carpeta "saved_games"
        if not os.path.exists("saved_games"):
            os.makedirs("saved_games")
        with open(f"saved_games/game_epoch_{epoch + 1}.pgn", "w") as pgn_file:
            exporter = chess.pgn.FileExporter(pgn_file)
            pgn_game.accept(exporter)
        
        print(f"Epoch {epoch + 1}/{epochs} terminado. Evaluación final: {evaluations_list[-1]}")
        print(f"Partida guardada en 'saved_games/game_epoch_{epoch + 1}.pgn'.")

Parte mas compleeja del codigo y es en la que entrenamos nuestra red neuronal a base de enfrentarla contra Stockfish.

Primero definimos un parametro importante para stockfish que es la profundida de busqueda que manejara el 15 hace referencia a la canttidad de movimientos por delante que el motor intentara mantenerse 

El primer bucle for es el define el usuario y se iterara la cantidad de veces que nosotros creamos necesarias (cada epoca es una partida) cada vez que este bucle se itere se reiniciara el tablero  , el ganador y si el estado de juego es terminado.

Luego tenemos el bucle while el cual estara reproduciendose siempre que la partida siga en juego.
    Lo primero que hacemos es activar el modelo entrenamiento del modelo haciendo que cosas como el dropout se activen 
    luego transformamos el estado del tablero y se lo damos a la red neuronal para que haga sus prediccion , la respuesta sera la probabilidad de los diferentes movimientos segun las caracteristicas del tablero esta respuesta se da en forma de tensor

Ahora obtenemos y procesamos los movimientos legales y asociamos esas predicciones de la red con los movimientos

Una vez hecho eso podemos buscar cual es el movimiento con mayor probabilidad y seleccionarlo para mover el tablero.
    Luego de cada movimiento nos fijamos si el juego a terminado.

Ahora esperamos a ver cual es la respuesta de Stockfish para luego evaluar que tan bueno fue nuestro movimiento y ajustar pesos, 
actualmente los pesos se ajustan luego de cada partida

Este que acabo de explicar se repite hasta que el juego a terminado y asi durante la cantidad de epocas que vayamos a usar 

In [18]:
if __name__ == "__main__":
    # Ruta a tu ejecutable de Stockfish
    stockfish_path = "stockfish-windows-x86-64-avx2.exe"

    model = ChessCNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Cargar el modelo si existe
    model, optimizer, start_epoch = load_model(model, optimizer)

    # Entrenamiento contra Stockfish, continúa desde la última época
    train_against_stockfish_with_evaluation(model, optimizer, stockfish_path, epochs=200, start_epoch=start_epoch + 1)

Modelo cargado desde model_checkpoint.pth, continuar desde la época 120
Epoch 121/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_121.pgn'.
Epoch 122/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_122.pgn'.
Epoch 123/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_123.pgn'.
Epoch 124/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_124.pgn'.
Epoch 125/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_125.pgn'.
Epoch 126/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_126.pgn'.
Epoch 127/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_127.pgn'.
Epoch 128/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_128.pgn'.
Epoch 129/200 terminado. Evaluación final: -1000
Partida guardada en 'saved_games/game_epoch_129.pgn'.
E